In [ ]:
class CustomDL(Dataset):
  def __init__(self,x_dir,y_dir,db,transforms = None,target_transforms=None):
    self.data = pd.read_csv(db)
    self.x_dir = x_dir
    self.y_dir = y_dir
    self.transform = transforms
    self.target_transform = target_transforms

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,idx):
    img_path_x = os.path.join(self.x_dir,self.data.iloc[idx,1])
    img_x = pd.read_pickle(img_path_x)
    img_x = torch.tensor(img_x)
    if self.transform:
      img_x = self.transform(img_x)
    img_path_y = os.path.join(self.y_dir,self.data.iloc[idx,1])
    img_y = pd.read_pickle(img_path_y)
    img_y = torch.tensor(img_y)
    if self.target_transform:
      img_y = self.target_transform(img_y)
    return img_x,img_y

In [ ]:
#Arch1
class Net(nn.Module):
  def __init__(self,in_channels,out_channels):
    super().__init__()
    self.in_channel = in_channels
    self.out_channel = out_channels;
    self.conv1 = nn.Conv2d(self.in_channel,64,kernel_size=3,padding="same",device=device) #in = 1024*1024*3,out = 1024*1024*64 
    self.batchnorm1 = nn.BatchNorm2d(64,device = device)
    self.dropout = nn.Dropout2d(0.5)
    self.maxpool1 = nn.MaxPool2d(2,2)#out = 512*512*64
    self.conv2 = nn.Conv2d(64,128,kernel_size=3,padding="same",device=device)#out = 512*512*128
    self.batchnorm2 = nn.BatchNorm2d(128,device = device)
    self.maxpool2 = nn.MaxPool2d(2,2)#out = 256*256*128
    #self.copy1 = self.maxpool2
    self.conv3 = nn.Conv2d(128,256,kernel_size=3,padding = "same",device = device)#out = 256*256*256
    self.batchnorm3 = nn.BatchNorm2d(256,device = device)
    self.maxpool3 = nn.MaxPool2d(2,2) #out = 128*128*256
    #self.copy2 = self.maxpool3
    self.conv4 = nn.Conv2d(256,512,kernel_size=3,padding = "same",device = device) #out = 128*128*512
    self.batchnorm4 = nn.BatchNorm2d(512,device = device)
    self.conv5 = nn.Conv2d(512,256,kernel_size = 1,device = device) #out = 128*128*256
    self.batchnorm5 = nn.BatchNorm2d(256,device = device)
    self.dropout5 = nn.Dropout2d(0.5)
    self.conv6 = nn.Conv2d(256,128,kernel_size = 1,device = device) #out = 128*128*128
    self.batchnorm6 = nn.BatchNorm2d(128,device = device)
    self.conv7 = nn.Conv2d(128,3,kernel_size = 1,device = device) #out = 128*128*3
    
    
  def forward(self,x):
    x = F.selu(self.batchnorm1(self.conv1(x)))
    x = self.maxpool1(self.dropout(x))
    x = F.selu(self.batchnorm2(self.conv2(x)))
    x = self.maxpool2(self.dropout(x))
    x = F.selu(self.batchnorm3(self.conv3(x)))
    x = self.maxpool3(self.dropout(x))
    x = F.selu(self.batchnorm4(self.conv4(x)))
    x = self.dropout(x)
    x = F.selu(self.batchnorm5(self.conv5(x)))
    x = self.dropout(x)
    x = F.selu(self.batchnorm6(self.conv6(x)))
    x = self.dropout(x)
    x = F.selu(self.conv7(x))
    return x

In [ ]:
#Arch2
class Net1(nn.Module):
  def __init__(self,start_size,end_size):
    super().__init__()
    self.start_size = start_size
    self.end_size = end_size
    self.conv1 = nn.Conv2d(self.start_size,64,2,2,device=device) #out = 512*512*64
    self.batchnorm1 = nn.BatchNorm2d(64,device = device)
    self.dropout = nn.Dropout2d(0.5)
    self.conv2 = nn.Conv2d(64,128,2,2,device = device) #out = 256*256*128
    self.batchnorm2 = nn.BatchNorm2d(128,device = device)
    self.conv3 = nn.Conv2d(128,256,2,2,device = device) #out = 128*128*256
    self.batchnorm3 = nn.BatchNorm2d(256,device = device)
    self.convT1 = nn.ConvTranspose2d(256,512,2,2,device = device)
    self.batchnorm4 = nn.BatchNorm2d(512,device = device)
    self.conv4 = nn.Conv2d(512,128,1,1,device = device) #out = 256*256*128
    self.batchnorm5 = nn.BatchNorm2d(128,device = device)
    self.convT2 = nn.ConvTranspose2d(128,256,2,2,device = device) #out = 512*512*256
    self.batchnorm6 = nn.BatchNorm2d(256,device = device)
    self.conv5 = nn.Conv2d(256,64,1,1,device = device) #out = 512*512*64
    self.batchnorm7 = nn.BatchNorm2d(64,device = device)
    self.maxpool1 = nn.MaxPool2d(2,2) #out = 256*256*64
    self.conv6 = nn.Conv2d(64,32,1,1,device = device) #out = 256*256*32
    self.batchnorm8 = nn.BatchNorm2d(32,device = device)
    self.maxpool2 = nn.MaxPool2d(2,2)
    self.conv7 = nn.Conv2d(32,self.end_size,1,1,device = device)

  
  def forward(self,x):
    x = F.selu(self.batchnorm1(self.conv1(x)))
    x = self.dropout(x)
    x_copy1 = x
    x = F.selu(self.batchnorm2(self.conv2(x)))
    x = self.dropout(x)
    x_copy2 = x
    x = F.selu(self.batchnorm3(self.conv3(x)))
    x = self.dropout(x)
    x = F.selu(self.batchnorm4(self.convT1(x)))
    x = self.dropout(x)
    x = F.selu(self.batchnorm5(self.conv4(x)))
    x = self.dropout(x)
    x += x_copy2
    x = F.selu(self.batchnorm6(self.convT2(x)))
    x = self.dropout(x)
    x = F.selu(self.batchnorm7(self.conv5(x)))
    x = self.dropout(x)
    x += x_copy1
    x = F.selu(self.batchnorm8(self.conv6(self.maxpool1(x))))
    x = self.dropout(x)
    x = F.selu(self.conv7(self.maxpool2(x)))
    return x

In [ ]:
def my_loss(output,target):
  err = torch.square((target-output))
  err = torch.mean(err)
  return err

In [ ]:
def train_nn(optim,loss_fn,train_loader,val_loader,max_epochs):
  min_val_loss =  math.inf
  patience = 5
  for epoch in range(max_epochs):
    running_loss = 0
    running_val_loss = 0
    net.train()
    for (i,batch) in enumerate(train_loader,0):
      x_img,y_img= batch
      x_img = x_img.to(device)
      y_img = y_img.to(device)
      optim.zero_grad()

      output = net(x_img)
      loss = loss_fn(output,y_img)
      loss = torch.sqrt(loss)
      loss.backward()
      optim.step()

      running_loss += loss.item()
      if i % 100 == 50:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0
    
    net.eval()
    with torch.no_grad():
      for (i,batch) in enumerate(val_loader,0):
        x_img,y_img= batch
        x_img = x_img.to(device)
        y_img = y_img.to(device)

        output = net(x_img)
        loss = loss_fn(output,y_img)
        loss = torch.sqrt(loss)
        
        running_val_loss += loss.item()
        
    
    if (min_val_loss>running_val_loss):
      min_val_loss = running_val_loss
      patience = 5
    else:
      patience =  patience - 1
      if (patience == 0):
        print("Early stopping on epoch {}".format(epoch+1))
        break

    print("Complete validation loss for this epoch {} is {}".format(epoch+1,running_val_loss))

In [ ]:
net = Net1(3,3)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
dataset = CustomDL("/content/gdrive/MyDrive/Vocals_Accompain","/content/gdrive/MyDrive/Dict_y_data","/content/gdrive/MyDrive/database.csv")
train,validation = random_split(dataset,[390,13])
train_loader = DataLoader(train,batch_size=4,shuffle=True,num_workers=0)
val_loader = DataLoader(validation,batch_size = 4,shuffle=True,num_workers=0)

In [ ]:
train_nn(optimizer,my_loss,val_loader,10)

In [ ]:
def gen_res(x):
  output = model(x)
  og_data = output[:,:,1]
  audio_file = librosa.feature.inverse.mel_to_audio(og_data)
  return audio_file